<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [1]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Union

import stopwords as stopwords

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [2]:
import pandas as pd
from collections import defaultdict

# Load the dataset
df = pd.read_csv('data2.csv')

# Use 70% of the data
#df = df.iloc[:int(df.shape[0] * 0.7)]

# Replace NaN values with an empty string
df['title'].fillna('', inplace=True)
df['abstract'].fillna('', inplace=True)


print(df)

                                       paperId  \
0     40ea606185b59cd07b456cb1022d64bf41f5538d   
1     597c2d96c45e8ad83fc08e5d464d266b68f873ed   
2     afbf330f0180320deff12fe42ded4f087b4a1811   
3     fa57e1cbaa46211499749cc75172e9ced26ad539   
4     8780ce7c0d09d36d293def6be23e64b12c644169   
...                                        ...   
6180  dd837f59449129861086e5a96b4819a9a697c541   
6181  41156a34679d10200ad4bc5ba3fb391c484417ab   
6182  7cc4ecd09745b8aec00bfe07618db4d6be05cca2   
6183  188a93c03c4db17c487ccc9419864d341acf0bb9   
6184  812bf641028468bcfe95d31d515129728e067d99   

                                                  title  \
0     Analysis on ground surface in ultrasonic face ...   
1     Measuring SARS-CoV-2 neutralizing antibody act...   
2                  A variational model for disocclusion   
3     Helicoidally Arranged Polyacrylonitrile Fiber-...   
4     The Interaction of SIRT4 and Calreticulin duri...   
...                                          

In [67]:
# Define a list of stop words
stop_words =  set(stopwords.get_stopwords('english'))

def preprocess_text(text: Union[str, pd.Series]) -> Union[str, pd.Series]:
    # If the input is a Series
    if isinstance(text, pd.Series):
        return text.apply(preprocess_text)

    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join(ch for ch in text if ch.isalnum() or ch.isspace())
    # Remove stop words
    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

# Replace NaN values with an empty string
df['title'].fillna('', inplace=True)
df['abstract'].fillna('', inplace=True)

# Apply preprocessing to the 'title' and 'abstract' columns
df['title'] = preprocess_text(df['title'])
df['abstract'] = preprocess_text(df['abstract'])

df['document'] = df['title'] + ' ' + df['abstract']
# Tokenize the 'document' column
df['document'] = df['document'].apply(lambda x: x.split())

# Create a dictionary where the keys are the 'paperid' and the values are the tokens
corpus = pd.Series(df.document.values,index=df.paperId).to_dict()

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [4]:
def construct_positional_indexes(corpus: pd.DataFrame) -> dict:
    positional_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))

    # Insert words from the processed data into a trie and construct positional indexes and posting lists.
    for index, row in corpus.iterrows():
        doc_id = row['paperId']
        for section in ['title', 'abstract']:
            words = row[section].split()
            for position, word in enumerate(words):
                positional_index[word][doc_id][section].append(position)

    return positional_index


# Construct the positional index
positional_index = construct_positional_indexes(df)
#positional_index = construct_positional_indexes(corpus)



# Print positional indexes
for word, postings in positional_index.items():
    print(f'Word: {word}')
    for doc_id, positions in postings.items():
        print(f'\tDocument ID: {doc_id}')
        for section, pos_list in positions.items():
            print(f'\t\tSection: {section}, Positions: {pos_list}')


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Word: ca2atpase
	Document ID: c1daaadc3093149f179db0f6b6bf61b285d1589a
		Section: title, Positions: []
		Section: abstract, Positions: [138]
Word: dynamin
	Document ID: c1daaadc3093149f179db0f6b6bf61b285d1589a
		Section: title, Positions: []
		Section: abstract, Positions: [147]
Word: phosphatidylinositol3
	Document ID: c1daaadc3093149f179db0f6b6bf61b285d1589a
		Section: title, Positions: []
		Section: abstract, Positions: [148]
Word: naadpmediated
	Document ID: c1daaadc3093149f179db0f6b6bf61b285d1589a
		Section: title, Positions: []
		Section: abstract, Positions: [157]
Word: dynamindependent
	Document ID: c1daaadc3093149f179db0f6b6bf61b285d1589a
		Section: title, Positions: []
		Section: abstract, Positions: [161]
Word: pmvp
	Document ID: f9934cf6f4c192b8ae865914ffe51f82b642c62c
		Section: title, Positions: []
		Section: abstract, Positions: [21, 128, 164]
Word: mahalanobis
	Document ID: f9934cf6f4c192b8ae865914ffe51f82b642c62c
		Section: title, Positions: []
		Section: abstract, Po

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [5]:
def get_tf(token: str, positional_index: dict, doc_id: str) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """
    # Implement the calculation of term frequency (TF) for the token in the specified document.
    tf = 0
    if token in positional_index and doc_id in positional_index[token]:
        for section in ['title', 'abstract']:
            tf += len(positional_index[token][doc_id][section])
    return tf

# Assuming 'positional_index' is your positional index and '291c1274ee45bf78662ed60831ef1bdaf89bed94' is a document ID
print(get_tf('3phosphorylated', positional_index, '43e690d73e98c10bd6bd14585acd17778b79701c'))


2


In [6]:

def get_idf(token: str, positional_index: dict) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.

    Returns
    -------
    float
        The IDF of the given token.
    """
    # Implement the calculation of IDF for the token based on your document collection.
    docs_with_token = len(positional_index[token])
    total_docs = sum(len(positional_index[t]) for t in positional_index)
    idf = math.log(total_docs / (1 + docs_with_token))
    return idf

# Assuming 'positional_index' is your positional index
print(get_idf('analyzers', positional_index))


11.777377292550558


<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [7]:
def generate_tfidf_list(corpus: dict, positional_index: dict) -> list[dict[str, float]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """
    tfidf_list = []
    tf_values = {}
    idf_values = {}

    for doc_id, doc in corpus.items():

        tfidf_scores = {}
        for token in doc:
            if token not in tf_values:
                tf_values[token] = get_tf(token, positional_index, doc_id)
            if token not in idf_values:
                idf_values[token] = get_idf(token, positional_index)
            tfidf_scores[token] = tf_values[token] * idf_values[token]
        tfidf_list.append({doc_id: tfidf_scores})
    return tfidf_list

# Assuming 'corpus' is your preprocessed data and 'positional_index' is your positional index
tfidf = generate_tfidf_list(corpus, positional_index)
print(tfidf)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
def show_document_vector(vector: list[dict[str, float]], doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.

    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_id : int
        The document index for which to display the vector.

    Returns
    -------
    None
    """
    # Get the document's TF-IDF scores
    doc_tfidf_scores = list(vector[doc_index].values())[0]

    # Filter out terms with a TF-IDF score of 0
    vector_list = [(term, tfidf) for term, tfidf in doc_tfidf_scores.items() if tfidf > 0]

    # Sort the terms by their TF-IDF scores in descending order
    vector_list.sort(key=lambda x: x[1], reverse=True)

    # Calculate the length of the vector for normalization
    length = math.sqrt(sum(tfidf ** 2 for _, tfidf in vector_list))

    # Normalize the TF-IDF scores
    normalized = {term: tfidf / length for term, tfidf in vector_list}

    # Print the terms and their TF-IDF scores
    for term, tfidf in vector_list:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')

# Assuming 'tfidf' is your list of document vectors
show_document_vector(tfidf,len(tfidf)-1)


rankings        	23.554754585101115	(normalized: 0.5755234132254456)
approach        	18.881893326977522	(normalized: 0.46134939153959437)
bias            	17.226619408354704	(normalized: 0.420905374514198)
complex         	13.850694057261881	(normalized: 0.33841994364988354)
territorial     	12.182842400658721	(normalized: 0.2976686093621931)
university      	9.265071668574443	(normalized: 0.22637746664739655)
network         	6.391354345583415	(normalized: 0.15616269975617159)


<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [90]:
def search(query: str, max_result_count: int, method: str = 'ltc-lnc-tat', weight: float = 1.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    # Initialize the scores for each document
    scores = {}

    # Tokenize the query
    query_tokens = query.split()

    # Calculate scores for each document
    for doc in tfidf:
        doc_id = list(doc.keys())[0]
        tfidf_scores = list(doc.values())[0]

        if method == 'ltc-lnc-tat' or method == 'ltc-lnc-dat':
            # Calculate score based on tf-idf values
            scores[doc_id] = sum(tfidf_scores[token] for token in query_tokens if token in corpus[doc_id])

            # Check if tokens are in the same order as in the validation file
            doc_tokens = [token for token in corpus[doc_id] if token in query_tokens]
            for i in range(len(query_tokens) - 1):
                if query_tokens[i:i+2] in [doc_tokens[j:j+2] for j in range(len(doc_tokens) - 1)]:
                    scores[doc_id] *= weight

        elif method == 'okapi25':
            # Calculate Okapi25 score
            for term in query_tokens:
                if term in tfidf_scores:
                    idf = get_idf(term, positional_index)
                    tf = get_tf(term, positional_index, doc_id)
                    k1 = 1.2
                    b = 0.75
                    scores[doc_id] = idf * ((k1 + 1) * tf) / (k1 * ((1 - b) + b) + tf)

    # Sort the documents by their scores
    scores = {doc_id: score for doc_id, score in scores.items() if score != 0}

    sorted_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    #print(sorted_docs)
    # Return the top 'max_result_count' documents
    if max_result_count == -1:
        return [doc_paperId for doc_paperId, _ in sorted_docs]
    else:
        return [doc_paperId for doc_paperId, _ in sorted_docs[:max_result_count]]


<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [91]:
# محاسبه تعداد هر کلمه در تمام اسناد
# محاسبه تعداد کل کلمات
total_words = 0
for text in corpus.values():
    total_words += len(text)

# محاسبه تعداد هر کلمه در تمام اسناد
word_counts = {}
for text in corpus.values():
    for word in text:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

# محاسبه احتمال هر کلمه با تقسیم تعداد آن بر تعداد کل کلمات
unigram_model = {word: count / total_words for word, count in word_counts.items()}


def lm_search(query: str, max_result_count: int):
    # Split the query into words
    words = query.split()

    # Calculate the score for each document
    scores = {}
    for doc, text in corpus.items():
        score = 0
        for word in words:
            # Use smoothing to handle words not in the unigram model
            score += unigram_model.get(word, 0)
        scores[doc] = score

    # Sort the documents by score in descending order
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Get the top 'max_result_count' documents
    if max_result_count != -1:
        sorted_docs = sorted_docs[:max_result_count]

    # Return the documents with snippets
    results = []
    for doc, score in sorted_docs:
        snippet = corpus[doc][:100]  # Just an example, you should generate a real snippet here
        results.append((doc, snippet))

    return results


<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [108]:
class Evaluation:
    def __init__(self, actual, predicts):
        self.actual = actual
        self.predicts = predicts

    def evaluate(self, require_scores):
        for score in require_scores:
            if score == 'precision':
                print('Precision: ', self.precision())
            elif score == 'recall':
                print('Recall: ', self.recall())
            elif score == 'f1':
                print('F1 Score: ', self.f1())
            elif score == 'map':
                print('MAP: ', self.map())
            elif score == 'ndcg':
                print('NDCG: ', self.ndcg())
            elif score == 'mrr':
                print('MRR: ', self.mrr())

    def precision(self):
        tp = 0
        fp = 0
        for i in range(len(self.predicts)):
           for j in range(len(self.predicts[i])):
               if self.predicts[i][j] in self.actual[i]:
                      tp += 1
               else:
                      fp += 1
        if tp+fp==0:precisionn=-1
        else:  precision = tp / (tp + fp)


        return precision

    def recall(self):
        tp = 0
        fn = 0
        for i in range(len(self.predicts)):
           for j in range(len(self.predicts[i])):
              if self.predicts[i][j] in self.actual[i]:
                   tp += 1
              else:
                  fn += 1
           if tp+fn==0:recall=-1
           else:  recall = tp / (tp + fn)
        return recall

    def f1(self):
        precision = self.precision()
        recall = self.recall()
        if precision + recall != 0:
                f1_score = 2 * ((precision * recall) / (precision + recall))
        else:
                f1_score = -1
        return f1_score


    def map(self):
        ap = 0
        for i in range(len(self.predicts)):
            tp = 0
            fp = 0
            for j in range(len(self.predicts[i])):
                if self.predicts[i][j] in self.actual[i]:
                    tp += 1
                    ap += tp / (tp + fp)
                else:
                    fp += 1
        map_score = ap / len(self.predicts)
        return map_score

    def ndcg(self):
        dcg = 0
        idcg = 0
        for i in range(len(self.predicts)):
            for j in range(len(self.predicts[i])):
                if self.predicts[i][j] in self.actual[i]:
                    dcg += 1 / np.log2(j + 2)
                idcg += 1 / np.log2(j + 2)
        ndcg_score = dcg / idcg
        return ndcg_score

    def mrr(self):
        rr = 0
        for i in range(len(self.predicts)):
            for j in range(len(self.predicts[i])):
                if self.predicts[i][j] in self.actual[i]:
                    rr += 1 / (j + 1)
                    break
        mrr_score = rr / len(self.predicts)
        return mrr_score


In [109]:
import json

require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

# Read actuals and queries from validation.json file
with open('validation.json', 'r') as f:
    data = json.load(f)

actuals = list(data.values())
queries = list(data.keys())
processed_queries = [preprocess_text(key) for key in queries]

In [110]:
predicts = [search(query, 10, method='ltc-lnc-tat') for query in processed_queries]
eval_vsm = Evaluation(actuals, predicts)
eval_vsm.evaluate(require_scores)


Precision:  0.65
Recall:  0.65
F1 Score:  0.65
MAP:  5.913822751322752
NDCG:  0.6646695611458963
MRR:  0.75


In [111]:
predicts = [search(query, 10, method='okapi25') for query in queries]

eval_bm = Evaluation(actuals, predicts)
eval_bm.evaluate(require_scores)

Precision:  0.16666666666666666
Recall:  0.16666666666666666
F1 Score:  0.16666666666666666
MAP:  0.5682539682539682
NDCG:  0.1986198396304303
MRR:  0.19047619047619047


In [112]:
predicts = [lm_search(query, 10) for query in queries]

eval_lm = Evaluation(actuals, predicts)
eval_lm.evaluate(require_scores)

Precision:  0.0
Recall:  0.0
F1 Score:  -1
MAP:  0.0
NDCG:  0.0
MRR:  0.0
